In [ ]:
from datascience import *
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.simplefilter("ignore")

## New material

##### You need not worry about the functions in these blocks of code (today).
  - First block: we will go over how all of these work within the next two classes!
  - Second block: not important for our course.

In [ ]:
def distance(point1, point2):
    """The distance between two arrays of numbers."""
    return np.sqrt(np.sum((point1 - point2)**2))

def all_distances(training, point):
    """The distance between p (an array of numbers) and the numbers in row i of attribute_table."""
    attributes = training.drop('Class')
    def distance_from_point(row):
        return distance(point, np.array(row))
    return attributes.apply(distance_from_point)

def table_with_distances(training, point):
    """A copy of the training table with the distance from each row to array p."""
    return training.with_column('Distance', all_distances(training, point))

def closest(training, point, k):
    """A table containing the k closest rows in the training table to array p."""
    with_dists = table_with_distances(training, point)
    sorted_by_distance = with_dists.sort('Distance')
    topk = sorted_by_distance.take(np.arange(k))
    return topk

def majority(topkclasses):
    """1 if the majority of the "Class" column is 1s, and 0 otherwise."""
    ones = topkclasses.where('Class', are.equal_to(1)).num_rows
    zeros = topkclasses.where('Class', are.equal_to(0)).num_rows
    if ones > zeros:
        return 1
    else:
        return 0

def classify(training, p, k):
    """Classify an example with attributes p using k-nearest neighbor classification with the given training table."""
    closestk = closest(training, p, k)
    topkclasses = closestk.select('Class')
    return majority(topkclasses)

def show_closest(point):
    """point = array([x,y]) 
    gives the coordinates of a new point
    shown in red"""
    
    HemoGl = ckd.drop('White Blood Cell Count', 'Color')
    t = closest(HemoGl, point, 1)
    x_closest = t.row(0).item(1)
    y_closest = t.row(0).item(2)
    ckd.scatter('Hemoglobin', 'Glucose', group='Class')
    plt.scatter(point.item(0), point.item(1), color='red', s=30)
    plt.plot(make_array(point.item(0), x_closest), make_array(point.item(1), y_closest), color='k', lw=2);

- This code is super cool, but it does not hold any conceptual value.

In [ ]:
def plot_all_points(test_grid):
    test_grid.scatter('Hemoglobin', 'Glucose', color='red', alpha=0.4, s=30)

    plt.scatter(ckd.column('Hemoglobin'), ckd.column('Glucose'), c=ckd.column('Color'), edgecolor='k')

    plt.xlim(-2, 2)
    plt.ylim(-2, 2);
    
def classify_grid(training, test, k):
    c = make_array()
    for i in range(test.num_rows):
        # Run the classifier on the ith patient in the test set
        c = np.append(c, classify(training, make_array(test.row(i)), k))   
    return c

def plot_all_points_classified(test_grid):
    c = classify_grid(ckd.drop('White Blood Cell Count', 'Color'), test_grid, 1)
    test_grid = test_grid.with_column('Class', c).join('Class', color_table)
    test_grid.scatter('Hemoglobin', 'Glucose', group='Class', alpha=0.4, s=30)

    plt.scatter(ckd.column('Hemoglobin'), ckd.column('Glucose'), c=ckd.column('Color'), edgecolor='k')

    plt.xlim(-2, 2)
    plt.ylim(-2, 2);

### What type of variable are we trying to predict?

- Today, they will be categorical. We will be therefore doing a classification task.

#### Case study 1: hospital patients

In [ ]:
ckd = Table.read_table('ckd.csv').relabeled('Blood Glucose Random', 'Glucose')
ckd.sample(3).show(3)

In [ ]:
ckd.group('Class')

In [ ]:
ckd.scatter('Hemoglobin', 'Glucose', group='Class')

- We want to be able to way to predict the class of someone without having to plot & eye ball this graph every time.
    - One way to do this is to decide on a *classification threshold*.

In [ ]:
max_glucose_for_0 = ckd.where('Class',are.equal_to(0)).column('Glucose').max()
min_hemoglobin_for_0 = ckd.where('Class',are.equal_to(0)).column('Hemoglobin').min()

In [ ]:
max_glucose_for_0

In [ ]:
min_hemoglobin_for_0

In [ ]:
def classify_manually(hemoglobin, glucose):
    if hemoglobin < min_hemoglobin_for_0 or glucose > max_glucose_for_0:
        return 1
    else:
        return 0

In [ ]:
classify_manually(15, 100)

In [ ]:
classify_manually(10, 300)

#### Case study 2: classifiying currency (bills)

In [ ]:
banknotes = Table.read_table('banknote.csv')
banknotes

In [ ]:
banknotes.group('Class')

In [ ]:
banknotes.scatter('WaveletSkew', 'Entropy', group='Class')

In [ ]:
banknotes.scatter('WaveletVar', 'WaveletCurt', group='Class')

In [ ]:
ax = plt.figure(figsize=(8,8)).add_subplot(111, projection='3d')
ax.scatter(banknotes.column('WaveletSkew'), 
           banknotes.column('WaveletVar'), 
           banknotes.column('WaveletCurt'), 
           c=banknotes.column('Class'));

#### The Nearest Neighbor classifer

First, we convert our features into standard units.

In [ ]:
ckd = Table().with_columns(
    'Hemoglobin', standard_units(ckd.column('Hemoglobin')),
    'Glucose', standard_units(ckd.column('Glucose')),
    'White Blood Cell Count', standard_units(ckd.column('White Blood Cell Count')),
    'Class', ckd.column('Class')
)

In [ ]:
color_table = Table().with_columns(
    'Class', make_array(0, 1),
    'Color', make_array('darkblue', 'gold')
)
ckd = ckd.join('Class', color_table)

In [ ]:
ckd.scatter('Hemoglobin', 'Glucose', group='Class')

In [ ]:
alice = make_array(0, 0.95)
show_closest(alice)

#### Decision boundaries are a great visualization tool for classification

In [ ]:
alice = make_array(0, 0.95)
show_closest(alice)

The boundary (visualized as a grid of points) is created with this block of code. 

In [ ]:
x_array = make_array()
y_array = make_array()
for x in np.arange(-2, 2.1, 0.1):
    for y in np.arange(-2, 2.1, 0.1):
        x_array = np.append(x_array, x)
        y_array = np.append(y_array, y)

test_grid = Table().with_columns(
    'Hemoglobin', x_array,
    'Glucose', y_array
)

In [ ]:
plot_all_points(test_grid)

In [ ]:
plot_all_points_classified(test_grid)

**STOP**

### Prediction is making informed guesses (on incomplete information)!

**Task**: Split the data from each of our case studies into training and testing sets.

- The training set and testing set need to be determined at random.
- In the following code, we are *permuting* the rows randomly and then extracting a test subset and training subset using `np.take()`. 

#### Case study 1

In [ ]:
shuffled_ckd = ckd.sample(with_replacement=False)

training = shuffled_ckd.take(np.arange(79))
testing = shuffled_ckd.take(np.arange(79, 158))

In [ ]:
training.scatter('Hemoglobin', 'Glucose', group='Class')

#### Case study 2

- In the last example I "hard-coded" the halfway number (I knew what it was based on prior knowledge and so just entered the number). 

- That is somewhat inefficient, so I'm going to find out what it is (roughly) with code this time and save myself the energy.

In [ ]:
halfway = round(banknotes.num_rows/2)
halfway

In [ ]:
shuffled_banknotes = banknotes.sample(with_replacement=False)
training = shuffled_banknotes.take(np.arange(halfway))
testing = shuffled_banknotes.take(np.arange(halfway, banknotes.num_rows))

In [ ]:
testing

Here's a much more efficient way to do what we did in the previous two steps: use the built-in `Table.split()` function in the `datascience` library.

In [ ]:
training, testing = banknotes.split(k = round(banknotes.num_rows/2))

_______

**Task**: Classify one point from the banknotes testing set. 

In [ ]:
first_testing_banknote = banknotes.drop('Class').row(0)

In [ ]:
first_testing_banknote

In [ ]:
classify(training=training,p=first_testing_banknote,k=5)

## Review 

In [ ]:
def standard_units(arr):
    return (arr - np.average(arr))/np.std(arr)

def correlation(t, x, y):
    x_standard = standard_units(t.column(x))
    y_standard = standard_units(t.column(y))
    return np.average(x_standard * y_standard)

def slope(t, x, y):
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd

def intercept(t, x, y):
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - slope(t, x, y)*x_mean

def fitted_values(t, x, y):
    """Return an array of the regression estimates at all the x values"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a*t.column(x) + b

def residuals(t, x, y):
    """Return an array of all the residuals"""
    predictions = fitted_values(t, x, y)
    return t.column(y) - predictions

In [ ]:
def bootstrap_prediction(t, x, y, new_x, repetitions=2500):
    """ 
    Makes a 95% confidence interval for the height of the true line at new_x, 
    using linear regression on the data in t (column names x and y).
    Shows a histogram of the bootstrap samples and shows the interval
    in gold.
    """

    # Bootstrap the scatter, predict, collect
    predictions = make_array()
    for i in np.arange(repetitions):
        resample = t.sample()
        predicted_y = prediction_at(resample, x, y, new_x)
        predictions = np.append(predictions, predicted_y)

    # Find the ends of the approximate 95% prediction interval
    left = percentile(2.5, predictions)
    right = percentile(97.5, predictions)
    round_left = round(left, 3)
    round_right = round(right, 3)

    # Display results
    Table().with_column('Prediction', predictions).hist(bins=20)
    plots.xlabel('predictions at x='+str(new_x))
    plots.plot([left, right], [0, 0], color='yellow', lw=8);
    print('Approximate 95%-confidence interval for height of true line at x =', new_x)
    print(round_left, 'to', round_right, '( width =', round(right - left, 3), ')') 

In [ ]:
births = Table.read_table('baby.csv')
births.sample(3)

In [ ]:
# Preterm and postterm pregnancy cutoffs, according to the CDC
make_array(37 * 7, 42 * 7)

In [ ]:
births = births.where('Gestational Days', are.between(225, 325))

In [ ]:
births.scatter('Gestational Days', 'Birth Weight')

In [ ]:
average_x = np.average(births.column('Gestational Days'))
average_x

In [ ]:
lines = Table(['slope','intercept', 'at 210', 'at 300', 'at 320'])

for i in range(10):
    resample = births.sample()
    a = slope(resample, 'Gestational Days', 'Birth Weight')
    b = intercept(resample, 'Gestational Days', 'Birth Weight')
    lines.append([a, b, a * 210 + b, a * 300 + b, a * 320 + b])

for i in np.arange(lines.num_rows):
    line = lines.row(i)
    plt.plot([210, 320], [line.item('at 210'), line.item('at 320')], lw=1)
    plt.scatter(300, line.item('at 300'), s=30, zorder=3)